<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [4]:
print("Ready")


Ready


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [8]:
import pandas as pd
import piplite
from pyodide.http import pyfetch

# download the dataset directly into the environment
dataset_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

response = await pyfetch(dataset_url)
with open("spacex_launch_geo.csv", "wb") as f:
    f.write(await response.bytes())

# read the file
df = pd.read_csv("spacex_launch_geo.csv")
df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [9]:
print("Shape:", df.shape)
df.head()


Shape: (56, 13)


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [16]:
import folium

# Define an initial map centered around the average coordinates
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=4)

# Add a marker for each launch site
for lat, lon, launch_site in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['Launch Site']):
    folium.Marker(
        location=[lat, lon],
        popup=launch_site,
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(site_map)

# Display the map
site_map


In [18]:
site_map.save("site_map.html")


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [19]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [20]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [23]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [33]:
import folium
from folium.features import DivIcon

# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a folium map centered at NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Create a small yellow circle at NASA JSC
circle = folium.Circle(
    location=nasa_coordinate,
    radius=500,       # small radius
    color='yellow',
    fill=True,
    fill_color='yellow'
).add_child(folium.Popup('NASA Johnson Space Center'))

# Add a text label marker
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color: #d35400;"><b>NASA JSC</b></div>'
    )
)

# Add circle and marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

# Display the map inline
site_map


In [35]:
# Save the map to an HTML file so it displays correctly in a browser
site_map.save('launch_sites_map.html')
print("Map saved to launch_sites_map.html. Open this file in a browser to view the map.")


Map saved to launch_sites_map.html. Open this file in a browser to view the map.


In [36]:
import folium
from folium.features import DivIcon

# Start location: NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a Folium map
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Add a yellow circle for NASA JSC
circle = folium.Circle(
    location=nasa_coordinate,
    radius=1000,
    color='yellow',
    fill=True,
    fill_color='yellow',
    popup='NASA Johnson Space Center'
)
site_map.add_child(circle)

# Add a marker with a label
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color:yellow;"><b>NASA JSC</b></div>'
    )
)
site_map.add_child(marker)

# Save to HTML and display a message
site_map.save('launch_sites_map.html')
print("Map saved! Open 'launch_sites_map.html' in a browser to view it.")


Map saved! Open 'launch_sites_map.html' in a browser to view it.


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [38]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

from folium.features import DivIcon

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle and a Marker
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    
    # Add a yellow circle with a popup showing the launch site name
    folium.Circle(
        location=coord,
        radius=1000,
        color='#ffff00',  # yellow
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Add a marker with the launch site name as a label
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


Proximity to the Equator:

The launch sites in Florida are around 28.5° N, and Vandenberg is around 34.6° N.

So none are on or very close to the Equator. They are in the northern hemisphere, roughly 28–35° north.

Proximity to the coast:

Florida sites are right on the east coast of Florida.

Vandenberg is on the central coast of California.

So yes, all launch sites are near the coast, which makes sense for rocket safety and over-ocean flight pa

takeaway: launch sites are coastal, not equatorial.

In [40]:
# Task 2: Mark the success/failed launches for each site on the map
# Add success/failure markers
# Use green for success, red for failure
import folium
from folium.features import DivIcon

# Initialize the map centered around the USA
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Define a color function based on success (1) or failure (0)
def get_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Add a Circle and Marker for each launch
for lat, lon, site, launch_class in zip(spacex_df['Lat'], spacex_df['Long'], spacex_df['Launch Site'], spacex_df['class']):
    # Circle
    folium.Circle(
        location=[lat, lon],
        radius=500,
        color=get_marker_color(launch_class),
        fill=True,
        fill_opacity=0.7
    ).add_child(folium.Popup(f"{site} - {'Success' if launch_class==1 else 'Failure'}")).add_to(site_map)
    
    # Label Marker
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#000000;"><b>{site}</b></div>'
        )
    ).add_to(site_map)

# Save the map to HTML
site_map.save("launch_sites_success_failure_map.html")
print("Map saved to launch_sites_success_failure_map.html. Open this file in a browser to view it.")


Map saved to launch_sites_success_failure_map.html. Open this file in a browser to view it.


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [41]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [42]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [43]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Check the last few rows to confirm
spacex_df[['Launch Site', 'class', 'marker_color']].tail(10)


,Launch Site,class,marker_color
46,KSC LC-39A,1,green
47,KSC LC-39A,1,green
48,KSC LC-39A,1,green
49,CCAFS SLC-40,1,green
50,CCAFS SLC-40,1,green
51,CCAFS SLC-40,0,red
52,CCAFS SLC-40,0,red
53,CCAFS SLC-40,0,red
54,CCAFS SLC-40,1,green
55,CCAFS SLC-40,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [57]:
# Add marker_cluster to current site_map
# Start map centered on NASA JSC

# --- Task 2: Success/Failure markers on the map (working version) ---

from folium.plugins import MarkerCluster
import folium

# Create base map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add yellow circles and site labels
for idx, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='yellow',
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    # Site label
    site_map.add_child(
        folium.map.Marker(
            [row['Lat'], row['Long']],
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 12; color:orange;"><b>{row["Launch Site"]}</b></div>'
            )
        )
    )

# Create marker cluster for launches
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch, red=fail, green=success
for idx, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} | {'Success' if row['class']==1 else 'Failure'}",
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)

# Save map
site_map.save('/drive/DS0203EN/module_3/launch_sites_map_task2_clustered.html')
print("Map saved to launch_sites_map_task2_clustered.html. Open in browser to see clustered green/red markers.")


Map saved to launch_sites_map_task2_clustered.html. Open in browser to see clustered green/red markers.


In [58]:
# Save the current site_map to an HTML file
site_map.save("launch_sites_map.html")
print("Map saved to 'launch_sites_map.html'. Open this file in a browser to view all markers.")


Map saved to 'launch_sites_map.html'. Open this file in a browser to view all markers.


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [59]:
# TASK 3: Calculate the distances between a launch site to its proximities
import math

# Haversine function to calculate distance in km
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi/2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    distance = R * c
    return distance

# Sample landmarks (coastline approximations)
landmarks = {
    'Beach_FL': [28.583, -80.567],  # near CCAFS
    'Beach_CA': [34.633, -120.615]  # near VAFB
}

# Create new columns for distances
for landmark, coords in landmarks.items():
    launch_sites_df[f'Distance_to_{landmark}_km'] = launch_sites_df.apply(
        lambda row: haversine(row['Lat'], row['Long'], coords[0], coords[1]), axis=1
    )

# Display the updated DataFrame with distances
launch_sites_df


,Launch Site,Lat,Long,Distance_to_Beach_FL_km,Distance_to_Beach_CA_km
0,CCAFS LC-40,28.562302,-80.577356,2.513917,3826.229090
1,CCAFS SLC-40,28.563197,-80.576820,2.401722,3826.243292
2,KSC LC-39A,28.573255,-80.646895,7.876480,3819.441614
3,VAFB SLC-4E,34.632834,-120.610745,3825.981946,0.389691


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [63]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map



Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [64]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [65]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Example: Distance from each launch site to closest coastline
# Replace these coordinates with the ones you found using MousePosition
# For demonstration, we use approximate nearby coastline points

# Closest coastlines (example coordinates)
coastline_coords = {
    'CCAFS LC-40': (28.56367, -80.57163),  # FL coast near Cape Canaveral
    'CCAFS SLC-40': (28.56367, -80.57163), # FL coast near Cape Canaveral
    'KSC LC-39A': (28.5735, -80.6536),     # FL coast near Kennedy Space Center
    'VAFB SLC-4E': (34.6325, -120.6150)    # CA coast near Vandenberg
}

# Add a new column in launch_sites_df for distance to coast
launch_sites_df['Distance_to_Coast_km'] = launch_sites_df.apply(
    lambda row: calculate_distance(
        row['Lat'], row['Long'],
        coastline_coords[row['Launch Site']][0],
        coastline_coords[row['Launch Site']][1]
    ), axis=1
)

launch_sites_df


,Launch Site,Lat,Long,Distance_to_Beach_FL_km,Distance_to_Beach_CA_km,Distance_to_Coast_km
0,CCAFS LC-40,28.562302,-80.577356,2.513917,3826.229090,0.579758
1,CCAFS SLC-40,28.563197,-80.576820,2.401722,3826.243292,0.509744
2,KSC LC-39A,28.573255,-80.646895,7.876480,3819.441614,0.655502
3,VAFB SLC-4E,34.632834,-120.610745,3825.981946,0.389691,0.391147


In [66]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [71]:
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster, MousePosition
from math import sin, cos, sqrt, atan2, radians

# ----------------------------
# Helper function for distances
# ----------------------------
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# ----------------------------
# Launch site coordinates
# ----------------------------
launch_sites = {
    "CCAFS LC-40": [28.562302, -80.577356],
    "CCAFS SLC-40": [28.563197, -80.576820],
    "KSC LC-39A": [28.573255, -80.646895],
    "VAFB SLC-4E": [34.632834, -120.610745]
}

# ----------------------------
# Closest coastline points (example)
# ----------------------------
coastlines = {
    "CCAFS LC-40": [28.56367, -80.57163],
    "CCAFS SLC-40": [28.56367, -80.57163],
    "KSC LC-39A": [28.5732, -80.6500],
    "VAFB SLC-4E": [34.6330, -120.6110]
}

# ----------------------------
# Sample spacex_df: success/failure launches
# ----------------------------
# class=1 -> success (green), class=0 -> failure (red)
# For simplicity, let's simulate marker_color column
# In your lab, you already have spacex_df with class column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

# ----------------------------
# Initialize map
# ----------------------------
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# ----------------------------
# Add launch site circles & markers
# ----------------------------
for site, coord in launch_sites.items():
    # Circle
    folium.Circle(
        coord,
        radius=1000,
        color='#d35400',
        fill=True,
        fill_opacity=0.5,
        popup=folium.Popup(site)
    ).add_to(site_map)
    
    # Text label
    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site
        )
    ).add_to(site_map)

# ----------------------------
# Add success/failure markers using MarkerCluster
# ----------------------------
marker_cluster = MarkerCluster().add_to(site_map)

for idx, row in spacex_df.iterrows():
    folium.Marker(
        [row['Lat'], row['Long']],
        icon=folium.Icon(color='white', icon_color=row['marker_color']),
        popup=f"{row['Launch Site']} | {'Success' if row['class']==1 else 'Failure'}"
    ).add_to(marker_cluster)

# ----------------------------
# Add distances to coastline
# ----------------------------
for site, coord in launch_sites.items():
    coast = coastlines[site]
    distance = calculate_distance(coord[0], coord[1], coast[0], coast[1])
    
    # Marker at coastline
    folium.Marker(
        coast,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s KM</b></div>' % "{:.2f}".format(distance)
        )
    ).add_to(site_map)
    
    # PolyLine
    folium.PolyLine([coord, coast], color='blue', weight=2, opacity=0.6).add_to(site_map)

# ----------------------------
# Add MousePosition for coordinates
# ----------------------------
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# ----------------------------
# Save map to HTML (JupyterLite safe)
# ----------------------------
site_map.save("/mnt/data/launch_sites_map.html")
print("Map saved to /mnt/data/launch_sites_map.html. Open this file in a browser to see all markers, circles, and lines.")


<class 'FileNotFoundError'>: [Errno 44] No such file or directory: '/mnt/data/launch_sites_map.html'

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [75]:
import folium
from folium.features import DivIcon
from folium.plugins import MousePosition
from math import sin, cos, sqrt, atan2, radians

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # approximate radius of Earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# NASA Johnson Space Center as map center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add launch site circles and labels
for idx, row in launch_sites_df.iterrows():
    coord = [row['Lat'], row['Long']]
    folium.Circle(
        coord,
        radius=1000,
        color='yellow',
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['Launch Site']}"
    ).add_to(site_map)
    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Example coastline points (replace with actual coordinates if you measured with MousePosition)
coastline_points = {
    'FL': [28.56367, -80.57163],
    'CA': [34.632, -120.610]
}

# Draw lines and distance markers from launch sites to coastline points
for idx, row in launch_sites_df.iterrows():
    site_coord = [row['Lat'], row['Long']]
    if 'FL' in row['Launch Site']:
        coast_coord = coastline_points['FL']
    else:
        coast_coord = coastline_points['CA']
    # Polyline
    line = folium.PolyLine([site_coord, coast_coord], color='blue', weight=2, opacity=0.6)
    site_map.add_child(line)
    # Distance marker
    dist_km = calculate_distance(site_coord[0], site_coord[1], coast_coord[0], coast_coord[1])
    folium.Marker(
        coast_coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{dist_km:.2f} KM</b></div>'
        )
    ).add_to(site_map)

# Enable mouse position
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

# Save the map in the current directory
site_map.save("launch_sites_map.html")
print("Map saved as launch_sites_map.html. Open this file in a browser to view it.")




Map saved as launch_sites_map.html. Open this file in a browser to view it.


In [79]:
import folium
from folium.features import DivIcon

# Sample launch site (you can loop through all if needed)
launch_site = [28.562302, -80.577356]  # CCAFS LC-40
launch_site_name = "CCAFS LC-40"

# Sample closest point (e.g., coastline)
closest_point = [28.56367, -80.57163]
closest_point_name = "Closest Coastline"

# Calculate distance function
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # radius of earth in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

distance_km = calculate_distance(launch_site[0], launch_site[1],
                                 closest_point[0], closest_point[1])

# Create the map centered at launch site
site_map = folium.Map(location=launch_site, zoom_start=10)

# Add a marker for launch site
folium.Circle(
    launch_site,
    radius=500,
    color='yellow',
    fill=True,
    popup=launch_site_name
).add_to(site_map)

# Add a marker for closest point with distance label
folium.map.Marker(
    closest_point,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_km:.2f} KM</b></div>'
    )
).add_to(site_map)

# Draw a line between launch site and closest point
line = folium.PolyLine([launch_site, closest_point], color='blue', weight=2, opacity=0.6)
site_map.add_child(line)

# Save map to current directory
site_map.save("launch_sites_map.html")
print("Map saved to launch_sites_map.html. Open this file in a browser to view the map.")


Map saved to launch_sites_map.html. Open this file in a browser to view the map.


1. Are launch sites in close proximity to railways?
Yes, launch sites are reasonably close to railways for logistics, but not directly adjacent.

2. Are launch sites in close proximity to highways?
Yes, they are near highways to allow easy transport, while maintaining a safe buffer.

3. Are launch sites in close proximity to the coastline?
Yes, all launch sites are very close to the coast to allow rockets to launch safely over open water.

4. Do launch sites keep a certain distance away from cities?
Yes, launch sites are intentionally located away from major cities to minimize risk to populations.

Summary of findings:
SpaceX launch sites are strategically located near transportation routes and the coast for operational efficiency and safety, while keeping a safe distance from populated areas.

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
